# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 1 2022 8:56AM,240923,15,30009371,"Alliance Pharma, Inc.",Released
1,Jul 1 2022 8:56AM,240923,15,30009372,"Alliance Pharma, Inc.",Released
2,Jul 1 2022 8:56AM,240923,15,30009373,"Alliance Pharma, Inc.",Released
3,Jul 1 2022 8:56AM,240923,15,30009374,"Alliance Pharma, Inc.",Released
4,Jul 1 2022 8:56AM,240923,15,30009376,"Alliance Pharma, Inc.",Released
5,Jul 1 2022 8:56AM,240923,15,30009375,"Alliance Pharma, Inc.",Released
6,Jul 1 2022 8:52AM,240927,10,0085817517,ISDIN Corporation,Released
7,Jul 1 2022 8:52AM,240927,10,0085817524,ISDIN Corporation,Released
8,Jul 1 2022 8:52AM,240927,10,0085817523,ISDIN Corporation,Released
9,Jul 1 2022 8:52AM,240927,10,0085817525,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,240921,Released,1
35,240922,Released,21
36,240923,Released,6
37,240926,Released,1
38,240927,Released,20


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240921,NaN,NaN,1.0
240922,NaN,NaN,21.0
240923,NaN,NaN,6.0
240926,NaN,NaN,1.0
240927,NaN,NaN,20.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240818,0.0,1.0,0.0
240831,0.0,0.0,1.0
240847,28.0,1.0,0.0
240870,0.0,4.0,4.0
240873,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240818,0,1,0
240831,0,0,1
240847,28,1,0
240870,0,4,4
240873,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240818,0,1,0
1,240831,0,0,1
2,240847,28,1,0
3,240870,0,4,4
4,240873,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240818,,1,
1,240831,,,1
2,240847,28,1,
3,240870,,4,4
4,240873,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 1 2022 8:56AM,240923,15,"Alliance Pharma, Inc."
6,Jul 1 2022 8:52AM,240927,10,ISDIN Corporation
26,Jul 1 2022 8:51AM,240906,10,ISDIN Corporation
36,Jul 1 2022 8:48AM,240926,10,ISDIN Corporation
37,Jul 1 2022 8:47AM,240922,15,"Laser Pharmaceuticals, LLC"
58,Jul 1 2022 8:44AM,240921,19,"GCH Granules USA, Inc."
59,Jul 1 2022 8:39AM,240920,10,"Snap Medical Industries, LLC"
63,Jul 1 2022 8:38AM,240919,10,Emerginnova
68,Jul 1 2022 8:30AM,240903,10,"Snap Medical Industries, LLC"
72,Jul 1 2022 8:30AM,240910,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 1 2022 8:56AM,240923,15,"Alliance Pharma, Inc.",,,6
1,Jul 1 2022 8:52AM,240927,10,ISDIN Corporation,,,20
2,Jul 1 2022 8:51AM,240906,10,ISDIN Corporation,,,10
3,Jul 1 2022 8:48AM,240926,10,ISDIN Corporation,,,1
4,Jul 1 2022 8:47AM,240922,15,"Laser Pharmaceuticals, LLC",,,21
5,Jul 1 2022 8:44AM,240921,19,"GCH Granules USA, Inc.",,,1
6,Jul 1 2022 8:39AM,240920,10,"Snap Medical Industries, LLC",,,4
7,Jul 1 2022 8:38AM,240919,10,Emerginnova,,,5
8,Jul 1 2022 8:30AM,240903,10,"Snap Medical Industries, LLC",,,4
9,Jul 1 2022 8:30AM,240910,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 1 2022 8:56AM,240923,15,"Alliance Pharma, Inc.",6,,
1,Jul 1 2022 8:52AM,240927,10,ISDIN Corporation,20,,
2,Jul 1 2022 8:51AM,240906,10,ISDIN Corporation,10,,
3,Jul 1 2022 8:48AM,240926,10,ISDIN Corporation,1,,
4,Jul 1 2022 8:47AM,240922,15,"Laser Pharmaceuticals, LLC",21,,
5,Jul 1 2022 8:44AM,240921,19,"GCH Granules USA, Inc.",1,,
6,Jul 1 2022 8:39AM,240920,10,"Snap Medical Industries, LLC",4,,
7,Jul 1 2022 8:38AM,240919,10,Emerginnova,5,,
8,Jul 1 2022 8:30AM,240903,10,"Snap Medical Industries, LLC",4,,
9,Jul 1 2022 8:30AM,240910,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 1 2022 8:56AM,240923,15,"Alliance Pharma, Inc.",6,,
1,Jul 1 2022 8:52AM,240927,10,ISDIN Corporation,20,,
2,Jul 1 2022 8:51AM,240906,10,ISDIN Corporation,10,,
3,Jul 1 2022 8:48AM,240926,10,ISDIN Corporation,1,,
4,Jul 1 2022 8:47AM,240922,15,"Laser Pharmaceuticals, LLC",21,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 1 2022 8:56AM,240923,15,"Alliance Pharma, Inc.",6.0,NaN,NaN
1,Jul 1 2022 8:52AM,240927,10,ISDIN Corporation,20.0,NaN,NaN
2,Jul 1 2022 8:51AM,240906,10,ISDIN Corporation,10.0,NaN,NaN
3,Jul 1 2022 8:48AM,240926,10,ISDIN Corporation,1.0,NaN,NaN
4,Jul 1 2022 8:47AM,240922,15,"Laser Pharmaceuticals, LLC",21.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 1 2022 8:56AM,240923,15,"Alliance Pharma, Inc.",6.0,0.0,0.0
1,Jul 1 2022 8:52AM,240927,10,ISDIN Corporation,20.0,0.0,0.0
2,Jul 1 2022 8:51AM,240906,10,ISDIN Corporation,10.0,0.0,0.0
3,Jul 1 2022 8:48AM,240926,10,ISDIN Corporation,1.0,0.0,0.0
4,Jul 1 2022 8:47AM,240922,15,"Laser Pharmaceuticals, LLC",21.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2890893,53.0,6.0,0.0
15,722692,27.0,1.0,28.0
16,1445330,14.0,2.0,0.0
19,2168069,20.0,12.0,0.0
20,481764,5.0,4.0,0.0
22,722734,1.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2890893,53.0,6.0,0.0
1,15,722692,27.0,1.0,28.0
2,16,1445330,14.0,2.0,0.0
3,19,2168069,20.0,12.0,0.0
4,20,481764,5.0,4.0,0.0
5,22,722734,1.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,53.0,6.0,0.0
1,15,27.0,1.0,28.0
2,16,14.0,2.0,0.0
3,19,20.0,12.0,0.0
4,20,5.0,4.0,0.0
5,22,1.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,53.0
1,15,Released,27.0
2,16,Released,14.0
3,19,Released,20.0
4,20,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,22
Status,,,,,,
Completed,0.0,28.0,0.0,0.0,0.0,0.0
Executing,6.0,1.0,2.0,12.0,4.0,2.0
Released,53.0,27.0,14.0,20.0,5.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,22
0,Completed,0.0,28.0,0.0,0.0,0.0,0.0
1,Executing,6.0,1.0,2.0,12.0,4.0,2.0
2,Released,53.0,27.0,14.0,20.0,5.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,22
0,Completed,0.0,28.0,0.0,0.0,0.0,0.0
1,Executing,6.0,1.0,2.0,12.0,4.0,2.0
2,Released,53.0,27.0,14.0,20.0,5.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()